In [0]:
bronze_df=spark.read.table("bijucatalog.bijubronzeschema.bronze_facebook")
bronze_df=bronze_df.toDF(*[c.upper() for c in bronze_df.columns])

In [0]:
from pyspark.sql.functions import col, to_date

def clean_facebook_ads_silver(df):
    """
    Cleans and transforms Facebook Ads data from the Bronze table for Silver layer.
    """
    df_clean = (
        df
        .dropna(subset=["CURRENCY", "CAMPAIGN_ID", "CAMPAIGN_NAME", "ADSET_ID", 
                        "ADSET_NAME", "AD_ID", "AD_NAME", "DATE"])
        .dropDuplicates(["AD_ID", "DATE", "AD_NAME"])
        .withColumn("INGEST_DATE", to_date(col("INGEST_DATE"), "dd-MM-yyyy"))  #
    )
  
    return df_clean


In [0]:
# Writing to Silver
df_silver = clean_facebook_ads_silver(bronze_df)
#display(df_silver)
df_silver.write.format("delta").mode("overwrite").saveAsTable("bijucatalog.bijusilverschema.silver_facebook")

In [0]:
%sql
select * from bijucatalog.bijusilverschema.silver_facebook

In [0]:
#spark.sql("DROP TABLE IF EXISTS bijucatalog.bijusilverschema.silver_facebook")


In [0]:
%sql
select count(* ) from  bijucatalog.bijusilverschema.silver_facebook